In [1]:
import gym
import json
import datetime as dt
import numpy as np
import pandas as pd
import tensorflow as tf
import os
from gym import wrappers




In [2]:
from env.StockTradingEnv import StockTradingEnv
from networks.DQNac import MyModel
from networks.DQNac import StockActor, StockCritic

In [3]:
df = pd.read_csv('./data/AAPL.csv')
df = df.sort_values('Date')

# The algorithms require a vectorized environment to run
env = StockTradingEnv(df)

base_action = np.array([0,0.5])

obs = env.reset()
for i in range(5):
    action, _states = (base_action, None)
    obs, rewards, done, info = env.step(action)
    
obs=np.expand_dims(obs,axis=0)
obs=np.expand_dims(obs,axis=0)
#print(obs)


In [4]:
act = StockActor()
act_target = StockActor()
output_action = act.predict(obs)
output_action_tar = act_target.predict(obs)
print("output_action")
print(output_action)
print(" ")
print("output_action_tar")
print(output_action_tar)
print(" ")
output_action = act.predict(obs)
output_action_tar = act_target.predict(obs)
print("output_action2")
print(output_action)
print(" ")
print("output_action_tar2")
print(output_action_tar)
print(" ")

output_action
tf.Tensor([[0.0035115]], shape=(1, 1), dtype=float32)
 
output_action_tar
tf.Tensor([[-0.00637227]], shape=(1, 1), dtype=float32)
 
output_action2
tf.Tensor([[0.0035115]], shape=(1, 1), dtype=float32)
 
output_action_tar2
tf.Tensor([[-0.00637227]], shape=(1, 1), dtype=float32)
 


In [6]:

#model = MyModel(32, (1,4), (1,1))
#output=tf.squeeze(model(obs))

crit = StockCritic()
crit_tar = StockCritic()

output = crit.predict(obs,output_action)
output_tar = crit_tar.predict(obs,output_action)
print("output")
print(output)
print(" ")
print("output_tar")
print(output_tar)
print(" ")
output = crit.predict(obs,output_action)
output_tar = crit_tar.predict(obs,output_action)
print("output2")
print(output)
print(" ")
print("output_tar2")
print(output_tar)
print(" ")


output
tf.Tensor([[2.2185523e-05]], shape=(1, 1), dtype=float32)
 
output_tar
tf.Tensor([[0.00010049]], shape=(1, 1), dtype=float32)
 
output2
tf.Tensor([[2.2185523e-05]], shape=(1, 1), dtype=float32)
 
output_tar2
tf.Tensor([[0.00010049]], shape=(1, 1), dtype=float32)
 
